In [2]:
from readdata import read_images_inorder
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, cdist

training_data, training_labels = read_images_inorder('../Yale_Face_Database/Training/')
testing_data, testing_labels = read_images_inorder('../Yale_Face_Database/Testing/')

def get_A(data):
    # A.T @ A equals to np.cov()
    data_ = np.array(data)
    mean = np.mean(data_,axis=1)
    A = np.array([i-j for (i,j) in zip(data_, mean)])
    A /= np.sqrt(np.size(data_,axis=1)-1)
    return A.T

def get_between_class_discrimination(training_data):
    clustered_traind = training_data.reshape(15,9,45045)

    diff_btw_clus_traind = []
    for i in range(0, 15):
        for j in range(i+1, 15):
            diff_btw_clus_traind.append(clustered_traind[i] - clustered_traind[j])
    diff_btw_clus_traind = np.array(diff_btw_clus_traind)

    SB = np.zeros((9,9))
    A = np.zeros((45045,9))
    
#     for i in range(105):
#         SB += np.cov(diff_btw_clus_traind[i])
    for i in range(105):
        tmpA = get_A(diff_btw_clus_traind[i])
        A += tmpA
        SB += tmpA.T @ tmpA
        
    return SB, A

SB, SB_A = get_between_class_discrimination(training_data)
# SB : (105, 9, 45045)

def get_inner_class_discrimination(training_data):
    clustered_traind = training_data.reshape(15,9,45045)
    SW = np.zeros((9,9))
#     for i in range(15):
#         SW += np.cov(clustered_traind[i])
    A = np.zeros((45045,9))
    for i in range(15):
        tmpA = get_A(clustered_traind[i])
        A += tmpA
        SW += tmpA.T @ tmpA
        
    return SW, A

SW, SW_A = get_inner_class_discrimination(training_data)

A_BovW = SB_A/SW_A
print(A_BovW.shape)

# C = A @ A.T

# sort
vals , vecs = np.linalg.eig(np.linalg.inv(SW)@SB)
vecs = vecs[:,np.argsort(vals)]
vals = vals[np.argsort(vals)]
vecs = np.flip(vecs,axis=0)
vals = np.flip(vals)

v = A_BovW @ vecs[0]


def get_projected(xs,y):
    # projected x onto y
    # xs contains serveral x, projected x onto y, and return ys
    xs_ = np.array(xs)
    y_norm = np.dot(y,y)
    for x in xs_:
        x = y * np.dot(x,y) / y_norm
    return xs_

def k_nearest_neighbor(dists, k, training_labels):
    # dist: distance toward every other points
    # we only utilize (training_points, testing_points) pair
    # spanned by test_num X training_num
    label = np.array(training_labels)
    test_label = []
    for test_points in dists:
        idx = np.argpartition(test_points, k)
        test_label.append( np.argmax( np.bincount( label[idx[:k]] )))
    return test_label

# train_projected = get_projected(training_data, v) + get_projected(training_data, v2) + get_projected(training_data, v3)
# test_projected = get_projected(testing_data, v) + get_projected(testing_data, v2) + get_projected(testing_data, v3)

train_projected = get_projected(training_data, v)
test_projected = get_projected(testing_data, v)

# calculate the distance between each [training data points], [testing data points] pair
dist_testXtrain = cdist(test_projected, train_projected)

test_label = k_nearest_neighbor(dist_testXtrain, 8, training_labels)
print(test_label)

std_label = testing_labels
rank_list = []
for k in range(1,135):
    test_label = k_nearest_neighbor(dist_testXtrain, k, training_labels)
    cnt = 0
    for i in range(30):
        if test_label[i] == std_label[i]:
            cnt += 1
    rank_list.append(cnt)
print(rank_list)

(45045, 9)
[1, 2, 2, 2, 3, 3, 4, 1, 5, 5, 1, 6, 15, 7, 5, 8, 9, 5, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15]
[26, 25, 25, 24, 26, 25, 26, 24, 23, 24, 24, 24, 26, 26, 23, 23, 22, 23, 20, 22, 20, 19, 21, 19, 19, 19, 17, 18, 18, 18, 17, 14, 14, 14, 14, 14, 14, 13, 13, 13, 13, 13, 12, 13, 12, 11, 11, 11, 12, 12, 12, 13, 14, 13, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 11, 11, 9, 9, 8, 7, 7, 7, 7, 8, 8, 8, 7, 7, 7, 7, 7, 6, 6, 6, 5, 6, 8, 8, 9, 9, 8, 7, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 7, 6, 6, 6, 5, 4, 4, 4, 4, 4, 4, 4, 3, 2, 2, 1, 1, 1, 1, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2]
